
#### 1.1 How many goals were attempted by head
#### 1.2 How many matches had attendees more than 12000
#### 2.1 Shots list where goal was scored outside 16m box sorted by descending order
#### 2.2 Rank teams based on average attendance at home games 

In [ ]:
from MySqlStruct import *

@measure_execution_time
def MySqlExecTime():
    return MySql_connect()

print('----Loading MySql----')
result, execution_time = MySqlExecTime()  # Get both the result and execution time

# Unpack the values from the result
cursor,connection,df_shots,df_matches,df_players,df_teams = result

----Loading MySql----


In [ ]:
# print(df_shots.columns)
# print(df_shots.head(5))
# print(df_matches.columns)
# print(df_matches.head(5))
# print(df_players.columns)
# print(df_players.head(5))
# print(df_teams.columns)
# print(df_teams.head(5))


# 1.1 How many goals were attempted by head

In [ ]:
@measure_execution_time
def MySQLQ1_1():
    # Define the query
    query = "SELECT * FROM shots WHERE Outcome = 'Goal' AND `Body Part` = 'Head';"
    
    # Execute the query
    cursor.execute(query)
    
    # Fetch all the results
    result = cursor.fetchall()
    
    # Print the first result
    if result:
        print(result[0])
    
    # Print the number of records
    print(f"Number of rows in result: {len(result)}")
    print('Query 1.1')

# Assuming you have the db_connection already defined
result, execution_time = MySQLQ1_1()

(1, '4a1a9578', 'Goal', 8.0, 'Head', 'de515487')
Number of rows in result: 3963
Query 1.1
Execution time: 0.12903761863708496 seconds


# 1.2 How many matches had attendees more than 12000

In [ ]:
@measure_execution_time
def MySQLQ1_2():
    # Define the query
    query = "SELECT * FROM matches WHERE Attendance > 12000;"

    # Execute the query
    cursor.execute(query)

    # Fetch all the results
    result = cursor.fetchall()

    # Print the first result (if any)
    if result:
        print(result[0])

    # Print the number of rows in the result
    print(f"Number of rows in result: {len(result)}")

    print('Query 1.2')

# Call the function
result, execution_time = MySQLQ1_2()


(2, '2022-10-09', 11, 11, 'Sun', 24671, '04eea015', '922493f3', 2, 2, 59, 41, '/en/players/75f2c59f/Roberto-Pereyra', '/en/players/2a1beb34/Marten-de-Roon', 1.0, 2.1, 2.1, 1.0, '3-5-2', '3-4-1-2', '2022-10-09 15:00:00')
Number of rows in result: 9166
Query 1.2
Execution time: 0.06401538848876953 seconds


#### 2.1 Total Goals Scoared and Inside VS Outside 16m box

#### 2.2 Rank teams based on average attendance at home games

In [ ]:
@measure_execution_time
def MySQLQ2_2(cursor):
    # Query to get average attendance and count of matches by home_id
    query = """
    SELECT 
        m.home_id,
        AVG(m.Attendance) AS average_attendance,
        COUNT(*) AS matches_count,
        t.name AS team_name
    FROM matches m
    INNER JOIN teams t ON m.home_id = t.id
    GROUP BY m.home_id
    ORDER BY average_attendance DESC
    """
    
    cursor.execute(query)
    results_from_matches = cursor.fetchall()  # List of tuples
    
    # Convert the result to a list of dictionaries for easier handling in Python
    results_from_matches = [
        {
            'home_id': row[0],
            'average_attendance': row[1],
            'matches_count': row[2],
            'team_name': row[3]
        } 
        for row in results_from_matches
    ]
    
    # Convert to DataFrame for easy viewing and sorting
    df = pd.DataFrame(results_from_matches)
    df = df[['team_name', 'average_attendance', 'matches_count']]
    print(df.head(10))  # Display the top 10 rows

    # return results_from_matches

# Call the function and measure execution time
result, execution_time = MySQLQ2_2(cursor)


              team_name average_attendance  matches_count
0                Troyes         51691.4586            157
1            Valladolid         50702.6525            141
2  Leicester City Women         48031.5000              8
3                 Molde         47265.5404            161
4         Sudburgenland         46952.0690            145
5      FC Luzern Frauen         46577.0500            160
6            Lillestrom         44531.8039            153
7  Internazionale Women         41408.8421             19
8               Udinese         39081.3636             11
9              Jaguares         38754.3158             19
Execution time: 0.0420079231262207 seconds


In [ ]:
@measure_execution_time
def MySQLQ2_1():
    # SQL query to filter only goals (Outcome = 'Goal')
    query = """
    SELECT * FROM shots WHERE Outcome = 'Goal';
    """
    
    cursor.execute(query)
    results_from_shots = cursor.fetchall()

    print(results_from_shots)

    # Extract player IDs to fetch player names
    # player_ids = [doc['player_id'] for doc in results_from_shots]
    # player_ids_str = ','.join(map(str, player_ids))

    # # Query to get player names by IDs
    # player_info_query = f"""
    # SELECT id, name FROM players
    # WHERE id IN ({player_ids_str});
    # """
    # cursor.execute(player_info_query)
    # player_info = cursor.fetchall()

    # # Map player IDs to names
    # player_name_map = {player['id']: player['name'] for player in player_info}

    # # Merge player names into the results
    # for doc in results_from_shots:
    #     player_id = doc['player_id']
    #     doc['name'] = player_name_map.get(player_id, 'Unknown')

    # # Convert results to DataFrame
    # df = pd.DataFrame(results_from_shots)
    # df = df[['name', 'distance', 'outcome']]

    # # Print the final results
    # print(df.head(10))

    # return df


# Call the function and measure execution time
result, execution_time = MySQLQ2_1()



DatabaseError: 2014 (HY000): Commands out of sync; you can't run this command now